In [173]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion, Pipeline 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.validation import check_is_fitted
import glob

In [2]:
big_df = pd.DataFrame() 
    
for filename in glob.glob("./Rating_*.xlsx"):
#    connres.commit()
    print ("READING DATABASE: "+filename)
    df = pd.read_excel(open(filename,'rb'), sheet_name="Resultados",header = None) #Reading SABI Export without index  
    df.columns = ['id', 'nombre_x', 'nif', 'nombre', 'provincia', 'calle', 'telefono', 'web', 'desc_actividad', 'cnae', 'cod_consolidacion', 'rating_grade_h2', 'rating_grade_h1', 'rating_grade_h0', 'rating_numerico_h2', 'rating_numerico_h1', 'rating_numerico_h0', 'modelo_propension_h2', 'modelo_propension_h1', 'modelo_propension_h0', 'guo_nombre', 'guo_id_bvd', 'guo_pais', 'guo_tipo', 'estado_detallado', 'fecha_cambio_estado', 'fecha_constitucion', 'p10000_h0', 'p10000_h1', 'p10000_h2', 'p20000_h0', 'p20000_h1', 'p20000_h2', 'p31200_h0', 'p31200_h1', 'p31200_h2', 'p32300_h0', 'p32300_h1', 'p32300_h2', 'p40100_mas_40500_h0', 'p40100_mas_40500_h1', 'p40100_mas_40500_h2', 'p40800_h0', 'p40800_h1', 'p40800_h2', 'p49100_h0', 'p49100_h1', 'p49100_h2']
    df['h0_anio'] = 2017     
    df = df.fillna('')
    df=df.drop(df.index[0]) #Dropping SABI variable names.
    df['nif'] = df.nif.str.upper() #CONVERTING cif INTO UPPERCASE
    for partida in ['p10000_h0', 'p10000_h1', 'p10000_h2', 'p20000_h0', 'p20000_h1', 'p20000_h2', 'p31200_h0', 'p31200_h1', 'p31200_h2', 'p32300_h0', 'p32300_h1', 'p32300_h2', 'p40100_mas_40500_h0', 'p40100_mas_40500_h1', 'p40100_mas_40500_h2', 'p40800_h0', 'p40800_h1', 'p40800_h2', 'p49100_h0', 'p49100_h1', 'p49100_h2']:
#        print (partida,"ha sido convertido en numerico")
        df[partida] = pd.to_numeric(df[partida], errors='coerce').fillna(0)- 0.005
    df['nif_normalizado'] = df['nif'].str[-8:]    
    big_df = big_df.append(df, ignore_index=True)     

READING DATABASE: ./Rating_1.xlsx
READING DATABASE: ./Rating_3.xlsx
READING DATABASE: ./Rating_2.xlsx
READING DATABASE: ./Rating_5.xlsx
READING DATABASE: ./Rating_4.xlsx


In [7]:
df = big_df
df['target_status'] = [0 if i in ['Activa', ''] else 1 for i in df['estado_detallado']] # 0 si Activa, 1 si algo raro!

# _h0, _h1, _h2
# _h0: history 0, here h0 means the year 2017 (historia 0, aquí h0 significa el año 2017)
# _h1: history -1, here h1 means the year 2016 (historia -1, aquí h1 significa el año 2016)
# _h2: history -2, here h2 means the year 2015 (historia -2, aquí h2 significa el año 2015)

# Ebita Margin - Ebitda / Turn over (Ventas)
# p49100: Profit (Resultado del ejercicio)
# p40800: Amortization (Amortización) 
# p40100: Sales Turnover (Ingresos de Explotación)
# p40500: Other sales (Otros Ingresos)
df['ebitda_income'] = (df.p49100_h1+df.p40800_h1)/(df.p40100_mas_40500_h1)

In [8]:
# Total Debt / Ebita 
# p31200: Short Term Debt / Deuda a corto plazo
# p32300: Long Term Debt / Deuda a largo plazo
# p49100: Profit (Resultado del ejercicio)
# p40800: Amortization (Amortización) 
df['debt_ebitda'] =(df.p31200_h1 + df.p32300_h1) /(df.p49100_h1+df.p40800_h1) 

# rraa_rrpp: Financial leveraging / apalancamiento financiero 
# p10000: Total Assets / Total activos
# p20000: Own Capital / Patrimonio neto
df['rraa_rrpp'] = (df.p10000_h1 - df.p20000_h1) /df.p20000_h1

# Log of Operating Income
df['log_operating_income'] = np.log(df.p40100_mas_40500_h1)

/Users/ocon/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ocon/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
df_clean = df[['cnae','ebitda_income','debt_ebitda','rraa_rrpp','log_operating_income','target_status']].replace([np.inf, -np.inf], np.nan).dropna()
X = df_clean[['cnae', 'ebitda_income','debt_ebitda','rraa_rrpp','log_operating_income']]
y = df_clean['target_status']

In [11]:
#Categrical features to pass down the categorical pipeline 
categorical_features = ["cnae"]

#Numerical features to pass down the numerical pipeline 
numerical_features = ['ebitda_income','debt_ebitda','rraa_rrpp','log_operating_income']

In [165]:
class CNAE_Transformer(BaseEstimator, TransformerMixin ):   

    #Return self nothing else to do here
    def fit( self, X, y = None  ):
        return self
    
    #Transformer method we wrote for this transformer 
    def transform(self, X , y = None ):  
        X = X.copy()
        X.loc[:, "sector"] = X.cnae.str[:2]
        X.sector = X.sector.str.strip()
        X = X.replace({"sector":""}, "missing")
        return X

In [166]:
class Mean_Imputer(BaseEstimator, TransformerMixin ):   

    #Return self nothing else to do here
    def fit( self, X, y = None  ):
        return self
    
    #Transformer method we wrote for this transformer 
    def transform(self, X , y = None ):  
        numeric_column_names = X.select_dtypes(include =["float64", "int"]).columns
        X = X.copy()
        X[numeric_column_names] = X[numeric_column_names].fillna(X.mean())
        return X

In [188]:

class GroupNormalizer(BaseEstimator, TransformerMixin):
    '''
    Class used for imputing missing values in a pd.DataFrame using either mean or median of a group.
    
    Parameters
    ----------    
    group_cols : list
        List of columns used for calculating the aggregated value 
    target : str
        The name of the column to impute
    metric : str

    Returns
    -------
    X : array-like
        The array with imputed values in the target column
    '''
    def __init__(self, group_cols, target):
        
        self.group_cols = group_cols
        self.target = target
    
    def fit(self, X, y=None):
        
        assert pd.isnull(X[self.group_cols]).any(axis=None) == False, 'There are missing values in group_cols'
        
        impute_map = X.groupby(self.group_cols)[self.target].agg([np.mean, np.std]) \
                                                            .reset_index(drop=False)
        
        self.impute_map_ = impute_map.fillna(impute_map.median())
        
        return self 
    
    def normalizer(self, df, group):
        df[self.target]
        
    
    def transform(self, X, y=None):
        
        # make sure that the imputer was fitted
        check_is_fitted(self, 'impute_map_')
        
        X = X.copy()
        df_final = pd.DataFrame(columns = X.columns)
        for group, df in X.groupby("sector"):
            df_final = df_final.append(self.normalizer(df, group))
            
        
        return self.impute_map_

In [189]:
pp = Pipeline([("CNAE_Transformer", CNAE_Transformer()), ("Mean_Imputer", Mean_Imputer()), ("standarize", GroupNormalizer(["sector"], numeric_column_names))])
Z = pp.fit_transform(X)

----------

A partir de aquí borrador son borradores y pruebas

In [29]:
class Standarizer( BaseEstimator, TransformerMixin ):   

    #Return self nothing else to do here
    def fit( self, X, y = None  ):
        self.statistics_group = dict()
        self.statistics_total = dict()
        self.numeric_column_names = X.select_dtypes(include =["float64", "int"]).columns
        for group, df in X.groupby("sector"):
            self.statistics_group[group] = {
                                "mean":{cn: df[cn].mean() for cn in self.numeric_column_names},
                                "std": {cn: df[cn].std() for cn in self.numeric_column_names}                        
                                }

        self.statistics_total = { "mean":{cn: df[cn].mean() for cn in self.numeric_column_names},
                             "std": {cn: df[cn].std() for cn in self.numeric_column_names}                        
                            }       
        return self

    def normalize(self, x, group):
        value_normalized = (x - self.statistics_group[group]["mean"]) / (self.statistics_group[group]["std"])         
        return value_normalized
        
    
    #Transformer method we wrote for this transformer 
    def transform(self, X , y = None ):      
        X.apply(lambda x: self.normalize(X[self.numeric_column_names],X["sector"]), axis = 1)
        return X

In [17]:
#Custom Transformer that extracts columns passed as argument to its constructor 
class FeatureSelector( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self._feature_names ] 